In [1]:
import sys
import numpy
import time
import matplotlib
from matplotlib import pyplot
sys.path.append('../')
sys.path.append('../../../../redundant_calibration/code/')
print(numpy.version.version)

1.16.5


In [2]:
from SCAR.SkyModel import analytic_visibilities
from gain_variance_simulation import get_observations

from skymodel import SkyRealisation
from radiotelescope import RadioTelescope
from radiotelescope import beam_width

In [3]:
position_path = "../Data/MWA_Hexes_Coordinates.txt"
frequency_range = numpy.linspace(135, 165, 100) * 1e6

#n_sources = 100
#fluxes = numpy.random.uniform(0.1, 1, n_sources)
#l_coordinates = numpy.random.uniform( -1, 1, n_sources)
#m_coordinates = numpy.random.uniform( -1, 1, n_sources)

fluxes = numpy.array([100])
l_coordinates = numpy.array([0.4])
m_coordinates = numpy.array([0.2])

telescope = RadioTelescope(load=True, path=position_path)
source_population = SkyRealisation(sky_type='random')
#print(len(source_population.fluxes))
source_population = SkyRealisation(sky_type = 'point', fluxes = fluxes, l_coordinates = l_coordinates, 
                                   m_coordinates = m_coordinates)


In [4]:

t0 = time.perf_counter()
dumb = get_observations(source_population, telescope.baseline_table, frequency_range = frequency_range,
                           interpolation = 'numba')
t1 = time.perf_counter()
print(t1-t0)

0.6864743260666728


In [5]:
t0 = time.perf_counter()
analytic = get_observations(source_population, telescope.baseline_table, frequency_range = frequency_range,
                           interpolation = 'analytic')
t1 = time.perf_counter()
print(t1-t0)

0.04273730400018394


In [6]:
t0 = time.perf_counter()

splined = get_observations(source_population, telescope.baseline_table, frequency_range = frequency_range,
                           interpolation = 'spline')
t1 = time.perf_counter()
print(t1-t0)

Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379,

KeyboardInterrupt: 

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/ronniyjoseph/Software/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "../gain_variance_simulation.py", line 235, in get_observations_single
    antenna_response2, interpolation='spline')
  File "../generaltools.py", line 166, in visibility_extractor
    uv_coordinates, interpolation = interpolation)
  File "../generaltools.py", line 140, in uv_list_to_baseline_measurements
    real_component = interpolate.RectBivariateSpline(u_bin_center

In [ ]:
t0 = time.perf_counter()

regular = get_observations(source_population, telescope.baseline_table, frequency_range = frequency_range,
                           interpolation = 'linear')
t1 = time.perf_counter()
print(t1-t0)

In [ ]:
dumb_data = dumb[600, :]
smart_data = analytic[600, :]
regular_data = regular[600, :]

%matplotlib inline

figure = pyplot.figure(figsize = (10,4))
real_axes = figure.add_subplot(2,2,1)
imag_axes = figure.add_subplot(2,2,2)
amp_axes = figure.add_subplot(2,2,3)
phas_axes = figure.add_subplot(2,2,4)

real_axes.plot(frequency_range/1e6, numpy.real(dumb_data) , label = 'Brute Force')
real_axes.plot(frequency_range/1e6, numpy.real(smart_data) , label = 'Fast')
real_axes.plot(frequency_range/1e6, numpy.real(regular_data) , label = 'Regular')

#real_axes.scatter(numpy.real(analytic), numpy.real(splined) - numpy.real(analytic), label = 'Spline')

imag_axes.plot(frequency_range/1e6, numpy.imag(dumb_data), label = 'Brute Force')
imag_axes.plot(frequency_range/1e6, numpy.imag(smart_data), label = 'Fast')
imag_axes.plot(frequency_range/1e6, numpy.imag(regular_data) , label = 'Regular')

#imag_axes.scatter(numpy.imag(analytic), numpy.imag(splined), label = 'Spline')

amp_axes.plot(frequency_range/1e6, numpy.abs(dumb_data), label = 'Brute Force')
amp_axes.plot(frequency_range/1e6, numpy.abs(smart_data),label = 'Fast')
amp_axes.plot(frequency_range/1e6, numpy.abs(regular_data) , label = 'Regular')

#amp_axes.scatter(numpy.abs(analytic), numpy.abs(splined), label = 'Spline')

phas_axes.plot(frequency_range/1e6, numpy.angle(dumb_data), label = 'Brute Force')
phas_axes.plot(frequency_range/1e6, numpy.angle(smart_data), label = 'Fast')
phas_axes.plot(frequency_range/1e6, numpy.angle(regular_data) , label = 'Regular')

#phas_axes.scatter(numpy.angle(analytic), numpy.angle(splined), label = 'Spline')
figure.savefig("Comparison" +str(numpy.version.version)+".pdf")
real_axes.legend()

In [ ]:
a = numpy.zeros((55000,100), dtype = complex)
print(a.nbytes/1e9)

In [ ]:
print(ln(b))